Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Daria Mazur"
COLLABORATORS = ""

---

In [2]:
import numpy as np

# Уравнение теплопроводности

Начально-краевая задача для уравнения теплопроводности с постоянным коэффициентом в общем виде можно записать следующим образом:
$$
\begin{array}{l}
\dfrac{\partial u}{\partial t} = \alpha \dfrac{\partial^2 u}{\partial x^2} + f(x, t), \; t > 0, \; x \in (0, l_x), \\
\left. u \right|_{t=0} = u_0(x), \\
\left. u \right|_{x=0} = \mu_1(t), \\
\left. u \right|_{x=l_x} = \mu_2(t),
\end{array}$$
где $u(x, t)$ - функция температуры, $\alpha = const$ - коэффициент теплопроводности, $f(x, t)$ - функция источника. 

## Явная схема

Запишем разностное уравнение явной схемы:
$$\dfrac{y_i^{k+1} - y_i^{k}}{\tau} = \alpha \dfrac{y_{i+1}^{k} - 2 y_i^{k} + y_{i-1}^{k}}{h^2} + f_i^k,$$
где $\tau$ и $h$ - шаги по времени и пространству, $y_i^k$ - значение приближённого решения в точке $(i, k)$ сетки, $y_i^k$ - значение функции источника в той же точке сетки. 

Аппроксимируем начальное и граничные условия:
$$
\begin{array}{l}
y_i^0 = u_0(x_i), \; \forall i = \overline{0, N},\\
y_0^k = \mu_1(t_k), \\
y_N^k = \mu_2(t_k), \; \forall k > 0.
\end{array}$$

Запрограммируйте явную разностную схему решения начально-краевой задачи для однородного уравнения теплопроводности. Обратите внимание, что 
$$\exists \lim\limits_{t \rightarrow \infty} u(x, t) = u_\infty (x).$$
поэтому расчёт в какой-то момент следует остановить (считать до установления).

Во время проведения расчетов помните о том, что явная схема *условно* устойчива.

In [3]:
def heat_expl(init, bound1, bound2, alpha, lx, h, tau, tol=1e-3):
    """ Solve the heat equation `u_t = a*u_xx` for x in (0; lx) with an explicit scheme.
    
    Parameters
    ----------
    init : callable
       Initial condition
    bound1 : callable
       Boundary condition for x = 0
    bound1 : callable
       Boundary condition for x = lx
    alpha : float
       Thermal diffusivity
    h : float
       Spatial step
    tau : float
       Time step
    tol : float, optional
       Target tolerance.
       Stop iterations when the 2-norm of the difference between 
       solution on this time step and the next is less the tol.
       
    Returns
    -------
    t_end : ndarray, shape (n,)
       End time of calculation
    u_end : ndarray, shape (N,)
       Limit u_∞(x) (See above)
    """
    
    N = int(lx/h)+1
    u_next = np.zeros((N,))
    u_prev= np.zeros((N,))
    for i in range(N-1):
        u_prev[i] = init(h*i)
    u_prev[-1]= init(lx)
    j = 0
    while 1:
        u_next[0]=bound1(tau*(j+1))
        u_next[-1]=bound2(tau*(j+1))
        for i in range(1,N-1):
            u_next[i]=tau*alpha*(u_prev[i+1]-2*u_prev[i]+u_prev[i-1])/h**2+u_prev[i]
        if np.linalg.norm(u_prev-u_next)<tol*10**(-2):
            break
        j+=1
        u_prev = u_next.copy() 
    t_end=tau*(j+1)
    u_end=u_next                     
    return t_end, u_end

Протестируйте Вашу функцию.

In [4]:
from numpy.testing import assert_allclose

t_0, u_0 = heat_expl(lambda x: 0., lambda x: 1., lambda x: 1., 
                     alpha=1., lx=1., h=0.1, tau=0.005) 
assert_allclose(u_0, np.ones(11), atol=1e-2)

t_1, u_1 = heat_expl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.1, tau=0.005) 
assert_allclose(u_1, np.zeros(32), atol=1e-2)


Определите порядки точности схемы (по пространству и времени) на тестовой задаче. Для этого для каждой переменной ($t$ или $x$):

1. Сделайте несколько расчётов для разных значений шага (например, $h_0, \; 2 h_0, \; 4 h_0$).
2. В один и тот же момент времени $t_1$ найдите ошибку полученных решений. Для этого либо возьмите аналитическое решение задачи, либо сравните результат в конечный момент времени, например, с решением в момент времени $0.99 t_1$. Обратите внимание, что имеющуюся функцию `heat_expl` надо немного модифицировать.
3. Найдите отношения этих ошибок. Сопоставьте полученные величины с порядком аппроксимации схемы по данной переменной.

In [5]:
def new_heat_expl(init, bound1, bound2, alpha, lx, h, tau, t1, true_sol, tol=1e-3):
    N = int(lx/h)+1
    u_next = np.zeros((N,))
    u_prev= np.zeros((N,))
    for i in range(N-1):
        u_prev[i] = init(h*i)
    u_prev[-1]= init(lx)
    j = 0
    while 1:
        u_next[0]=bound1(tau*(j+1))
        u_next[-1]=bound2(tau*(j+1))
        for i in range(1,N-1):
            u_next[i]=tau*alpha*(u_prev[i+1]-2*u_prev[i]+u_prev[i-1])/h**2+u_prev[i]
        if np.linalg.norm(u_prev-u_next)<tol:
            break 
        j+=1
        if j==t1:
            err = np.linalg.norm(u_next-true_sol)
        u_prev = u_next.copy() 
    t_end=(j+1)*tau
    u_end=u_next                     
    return t_end, u_end, err

In [6]:
t1 = 50
tau = 0.005
print("Зафиксируем момент времени t =", tau*t1)
t_0, u_0, err_0 = new_heat_expl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.1, tau=tau, t1=t1, true_sol=np.zeros(32))
t_2, u_2, err_2 = new_heat_expl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.2, tau=tau, t1=t1, true_sol=np.zeros(16))
t_4, u_4, err_4 = new_heat_expl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.4, tau=tau, t1=t1, true_sol=np.zeros(8))
print("Ошибка для шага h=0.1:", err_0)
print("Ошибка для шага h=0.2:", err_2)
print("Ошибка для шага h=0.4:", err_4)

Зафиксируем момент времени t = 0.25
Ошибка для шага h=0.1: 0.06938259520246938
Ошибка для шага h=0.2: 0.10019633335388985
Ошибка для шага h=0.4: 0.1573609260544599


In [7]:
print(err_2/err_0)
print(err_4/err_2)

1.4441133696642667
1.570525794578398


## Неявная схема 

Запишем разностное уравнение неявной схемы:
$$\dfrac{y_i^{k+1} - y_i^{k}}{\tau} = \alpha \dfrac{y_{i+1}^{k+1} - 2 y_i^{k+1} + y_{i-1}^{k+1}}{h^2} + f_i^{k+1}.$$

Аппроксимировать начальное и граничные условия будем так же, как в случае явной схемы.

Запрограммируйте явную разностную схему решения начально-краевой задачи для однородного уравнения теплопроводности. Для решения системы линейных уравнений используйте встроенные функции `scipy`.

In [8]:
from scipy.linalg import solve_banded

In [9]:
def heat_impl(init, bound1, bound2, alpha, lx, h, tau, tol=1e-3):
    """ Solve heat equation u_t = a*u_xx for x in (0; lx) with implicit scheme
    
    Parameters
    ----------
    init : callable
       Initial condition
    bound1 : callable
       Boundary condition for x = 0
    bound1 : callable
       Boundary condition for x = lx
    alpha : float, optional
       Thermal diffusivity
    h : float
       Spatial step
    tau : float
       Time step
    tol : float, optional
       Target tolerance.
       Stop iterations when the 2-norm of the difference between 
       solution on this time step and the next is less the tol.
       
    Returns
    -------
    t_end : ndarray, shape (n,)
       End time of calculation
    u_end : ndarray, shape (N,)
       Limit u_∞(x) (See above)
    """
    
    N = int(lx/h)+1
    u_next = np.zeros((N,))
    u_prev= np.zeros((N,))
    for i in range(N-1):
        u_prev[i] = init(h*i)
    u_prev[-1]= init(lx)
    c = alpha*tau/h**2
    A = np.zeros((3, N))
    A[0,2:]=-c
    A[1][1:-1]=1+2*c
    A[1,0] = 1.
    A[1,-1] =1.
    A[-1][:-2]=-c
    j = 0
    while 1:
        b = u_prev.copy()
        b[0]=bound1(tau*(j+1))
        b[-1]=bound2(tau*(j+1))
        u_next = solve_banded((1,1),A, b)
        #print(np.around(b, 2),np.around(u_next, 2))
        if np.linalg.norm(u_prev-u_next)<tol*0.01:
            break
        j+=1
        u_prev = u_next.copy() 
    t_end=tau*(j+1)
    u_end=u_next                     
    return t_end, u_end

Протестируйте Вашу функцию.

In [10]:
from numpy.testing import assert_allclose

t_0, u_0 = heat_impl(lambda x: 0., lambda x: 1., lambda x: 1., 
                     alpha=1., lx=1., h=0.1, tau=0.005) 
assert_allclose(u_0, np.ones(11), atol=1e-2)

t_1, u_1 = heat_impl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.1, tau=0.005) 
assert_allclose(u_1, np.zeros(32), atol=1e-2)


Определите порядки точности схемы (по пространству и времени) на тестовой задаче. (см. выше)

In [11]:
def new_heat_impl(init, bound1, bound2, alpha, lx, h, tau, t1, true_sol, tol=1e-3):
    """ Solve heat equation u_t = a*u_xx for x in (0; lx) with implicit scheme
    
    Parameters
    ----------
    init : callable
       Initial condition
    bound1 : callable
       Boundary condition for x = 0
    bound1 : callable
       Boundary condition for x = lx
    alpha : float, optional
       Thermal diffusivity
    h : float
       Spatial step
    tau : float
       Time step
    tol : float, optional
       Target tolerance.
       Stop iterations when the 2-norm of the difference between 
       solution on this time step and the next is less the tol.
       
    Returns
    -------
    t_end : ndarray, shape (n,)
       End time of calculation
    u_end : ndarray, shape (N,)
       Limit u_∞(x) (See above)
    """
    
    N = int(lx/h)+1
    u_next = np.zeros((N,))
    u_prev= np.zeros((N,))
    for i in range(N-1):
        u_prev[i] = init(h*i)
    u_prev[-1]= init(lx)
    c = alpha*tau/h**2
    A = np.zeros((3, N))
    A[0,2:]=-c
    A[1][1:-1]=1+2*c
    A[1,0] = 1.
    A[1,-1] =1.
    A[-1][:-2]=-c
    j = 0
    while 1:
        b = u_prev.copy()
        b[0]=bound1(tau*(j+1))
        b[-1]=bound2(tau*(j+1))
        u_next = solve_banded((1,1),A, b)
        #print(np.around(b, 2),np.around(u_next, 2))
        if np.linalg.norm(u_prev-u_next)<tol*0.01:
            break
        j+=1
        if j==t1:
            err = np.linalg.norm(u_next-true_sol)
        u_prev = u_next.copy() 
    t_end=tau*(j+1)
    u_end=u_next                     
    return t_end, u_end, err

In [12]:
t1 = 50
tau = 0.005
print("Зафиксируем момент времени t =", tau*t1)
t_0, u_0, err_0 = new_heat_impl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.1, tau=tau, t1=t1, true_sol=np.zeros(32))
t_2, u_2, err_2 = new_heat_impl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.2, tau=tau, t1=t1, true_sol=np.zeros(16))
t_4, u_4, err_4 = new_heat_impl(lambda x: np.sin(4.*x), lambda x: 0., lambda x: 0., 
                     alpha=1., lx=np.pi, h=0.4, tau=tau, t1=t1, true_sol=np.zeros(8))
print("Ошибка для шага h=0.1:", err_0)
print("Ошибка для шага h=0.2:", err_2)
print("Ошибка для шага h=0.4:", err_4)

Зафиксируем момент времени t = 0.25
Ошибка для шага h=0.1: 0.08954447140207836
Ошибка для шага h=0.2: 0.10936596985039083
Ошибка для шага h=0.4: 0.16639122073205848


In [13]:
print(err_2/err_0)
print(err_4/err_2)

1.221359265825678
1.521416771228531
